### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** if you having trouble displaying the maps try running `jupyter nbextension enable --py gmaps` in your environment and retry.

* Create a heat map that displays the humidity for every city from the part I of the homework.

  ![heatmap](Images/heatmap.png)

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

  ![hotel map](Images/hotel_map.png)

As final considerations:

* Create a new GitHub repository for this project called `API-Challenge` (note the kebab-case). **Do not add to an existing repo**
* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* For Part I, you must include a written description of three observable trends based on the data.
* You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
* For max intensity in the heat map, try setting it to the highest humidity found in the data set.

# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "../WeatherPy/output_data/cities.csv"
city_df = pd.read_csv(cities_csv)
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Carnarvon,0,AU,1579794362,78,-24.87,113.63,77.00,20.80
1,Hilo,20,US,1579794362,63,19.73,-155.09,66.20,8.05
2,George Town,20,MY,1579794069,79,5.41,100.34,84.20,6.93
3,Lata,40,PT,1579794362,76,40.16,-8.33,57.00,5.82
4,Mar del Plata,40,AR,1579794101,51,-38.00,-57.56,91.99,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
max_humidity = city_df['Humidity'].astype(int).max() # For max intensity in the heat map, try setting it to the highest humidity found in the data set.
max_humidity

100

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Set Lat/Lng and Humidity as variables
coordinates = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)

# Create initial map
fig = gmaps.figure()

# Create a "heatmap layer" that will represent the range of humitidy at the coordinate provided
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=1)
# dissipating: Specifies whether heatmaps dissipate on zoom. When dissipating is false the radius of influence 
# increases with zoom level to ensure that the color intensity is preserved at any given geographic location. Defaults to true.

# maxIntensity: The maximum intensity of the heatmap. By default, heatmap colors are dynamically scaled according to the 
# greatest concentration of points at any particular pixel on the map. This property allows you to specify a fixed maximum. 
# Setting the maximum intensity can be helpful when your dataset contains a few outliers with an unusually high intensity.

# radius: The radius of influence for each data point, in pixels.

# Add heatmap layer and display figure
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = city_df.copy() # df name pulled from "do not change" code given below but was not actually needed to be called this.
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 60] # Not too cold
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80] # Not too hot
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10] # Not too windy
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] <= 10] # Not too cloudy
narrowed_city_df = narrowed_city_df.dropna() # Meets requirement "Drop any rows will null values."
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,Kapaa,1,US,1579794204,88,22.08,-159.32,69.80,5.82
1,34,Cabo San Lucas,5,MX,1579794347,87,22.89,-109.91,63.00,4.70
2,78,Jiazi,0,CN,1579794426,90,22.88,116.07,66.56,4.52
3,141,Kahului,1,US,1579794412,82,20.89,-156.47,64.99,4.70
4,145,Port Alfred,1,ZA,1579794417,82,-33.59,26.89,70.00,3.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_city_df # Meets requirement "Store into variable named hotel_df."
hotel_df["Hotel Name"] = np.nan # Creates placeholder for hotel name to be filled later
hotel_df = hotel_df.drop(columns='index') #deletes un-needed column from output above
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Kapaa,1,US,1579794204,88,22.08,-159.32,69.80,5.82,NaN
1,Cabo San Lucas,5,MX,1579794347,87,22.89,-109.91,63.00,4.70,NaN
2,Jiazi,0,CN,1579794426,90,22.88,116.07,66.56,4.52,NaN
3,Kahului,1,US,1579794412,82,20.89,-156.47,64.99,4.70,NaN
4,Port Alfred,1,ZA,1579794417,82,-33.59,26.89,70.00,3.00,NaN


In [7]:
# Create empty list for hotel names
hotels = []
#Create index variable as record indexer in "for loop"
index = 0

# "for loop" to search through city coordinates from the hotels.df
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][index])+' , '+str(hotel_df['Lng'][index])
    index+=1
    target_radius = 5000 # Meets requirement "Set parameters to search for hotels with 5000 meters."
    target_type = 'lodging' # hotel not listed https://developers.google.com/places/supported_types 
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Setup error checking for API returns    
    try:
        response = requests.get(base_url, params=params) # Meets requirement "Hit the Google Places API for each city's coordinates."
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name']) # Meets "first result" part of requirement "Store the first Hotel result into the DataFrame."
        
    except IndexError:
        hotels.append(np.nan) # if it can't find lodging within 5000M of the target coordinates it put the NaN

# append dataframe with hotel names
hotel_df['Hotel Name'] = hotels # Meets "into DataFrame" part of requirement "Store the first Hotel result into the DataFrame."
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Kapaa,1,US,1579794204,88,22.08,-159.32,69.80,5.82,Kauai Shores Hotel
1,Cabo San Lucas,5,MX,1579794347,87,22.89,-109.91,63.00,4.70,Hotel Tesoro Los Cabos
2,Jiazi,0,CN,1579794426,90,22.88,116.07,66.56,4.52,Hotel Jin Hao
3,Kahului,1,US,1579794412,82,20.89,-156.47,64.99,4.70,Maui Seaside Hotel
4,Port Alfred,1,ZA,1579794417,82,-33.59,26.89,70.00,3.00,The Halyards Hotel
5,Babai,0,IN,1579794545,49,22.70,77.93,62.01,6.73,G loading service babai
6,Kaeo,0,NZ,1579794602,100,-35.10,173.78,61.00,7.90,Bushido Lodge
7,Suclayin,10,PH,1579794605,87,15.76,121.55,61.02,3.15,Costa Pacifica
8,Makakilo City,1,US,1579794663,100,21.35,-158.09,66.99,2.59,Marriott's Ko Olina Beach Club
9,Aswan,0,EG,1579794722,22,24.09,32.91,62.40,9.42,Nuba Nile


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()] # Changed to hotel_df instead of narrowed_cities_df
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers) #Meets requirement "Plot markers on top of the heatmap."

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))